## Feature Extraction Notebook

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.decomposition import PCA

### Feature Extraction

In [2]:
path = './ClassifyTripType/'
tr_path = path + 'train.csv'
te_path = path + 'test.csv'

In [3]:
def feature_extract(path):
    df = pd.read_csv(path, dtype={'Upc': object})
    
    # Deal with missing data
    nullDD = df['DepartmentDescription'].isnull()
    df.loc[nullDD, 'FinelineNumber'] = -2
    df.loc[nullDD, 'DepartmentDescription'] = 'UNKNOWN'
    df['FinelineNumber'].fillna(-1, inplace=True)
    df.loc[df.DepartmentDescription=='MENSWEAR', 'DepartmentDescription'] = 'MENS WEAR'
    
    # Extract features
    
    # grouped features
    grouped = df.groupby('VisitNumber')
    RI = grouped.ScanCount.apply(lambda x: 1 if any(x < 0) else 0)  # whether a product is returned
    RI.name = 'ReturnIndicator'
    NumDD = grouped.DepartmentDescription.apply(lambda x: len(x.unique()))  # Number of DD count
    NumDD.name = 'CountDD'
    NumUPC = grouped.Upc.apply(lambda x: len(x.unique()))  # kinds of UPC count
    NumUPC.name = 'CountUPC'
    
    # weekday
    WD = df[['VisitNumber', 'Weekday']].drop_duplicates(subset='VisitNumber')
    WD = pd.get_dummies(WD)
    WD.columns = ['VisitNumber', 'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
    WD.set_index('VisitNumber', inplace=True)
    
    # DD_sum
    DD = df.pivot_table('ScanCount', ['VisitNumber'], ['DepartmentDescription'], aggfunc=np.sum).fillna(0)
    
    # FN_sum
    FN = df.pivot_table('ScanCount', ['VisitNumber'], ['FinelineNumber'], aggfunc=np.sum).fillna(0)
    FN.columns = ['FN' + str(x) for x in np.array(FN.columns).astype(int)]
    
    Feature = pd.concat([RI, NumDD, NumUPC, WD, DD, FN], axis=1, keys=['RI', 'NumDD', 'NumUPC', 'WD', 'DD', 'FN'])
    
    del grouped, RI, NumDD, NumUPC, WD, DD, FN
    
    if 'TripType' in df.columns:
        # Extract training set labels
        trainLabel = df[['VisitNumber', 'TripType']].drop_duplicates().set_index('VisitNumber')
        return Feature, trainLabel
    else:
        return Feature

In [4]:
trainFeature, trainLabel = feature_extract(tr_path)
print('training set done')
testFeature = feature_extract(te_path)
print('test set done')

training set done
test set done


In [5]:
allFeature = pd.concat([trainFeature, testFeature], keys = ['train', 'test']).fillna(0)
del trainFeature, testFeature
print('concat done')

concat done


In [10]:
allFeature

DD                                \
DepartmentDescription 1-HR PHOTO ACCESSORIES AUTOMOTIVE BAKERY   
      VisitNumber                                                
train 5                      0.0         0.0        0.0    0.0   
      7                      0.0         0.0        0.0    0.0   
      8                      0.0         0.0        0.0    0.0   
      9                      0.0         0.0        0.0    0.0   
      10                     0.0         0.0        0.0    0.0   
      11                     0.0         0.0        0.0    0.0   
      12                     0.0         0.0        0.0    0.0   
      15                     0.0         0.0        0.0    0.0   
      17                     0.0         0.0        0.0    0.0   
      19                     0.0         1.0        0.0    0.0   
      20                     0.0         0.0        0.0    0.0   
      23                     0.0         0.0        0.0    0.0   
      25                     0.0         0.0        0.0    0.0   
      26                     0.0         0.0        0.0    0.0   
      28                     0.0         0.0        0.0    2.0   
      29                     0.0         0.0        0.0    0.0   
      30                     0.0         0.0        0.0    0.0   
      31                     0.0         0.0        0.0    1.0   
      32                     0.0         0.0        0.0    0.0   
      33                     0.0         0.0        0.0    0.0   
      40                     0.0         0.0        0.0    0.0   
      41                     0.0         0.0        0.0    0.0   
      42                     0.0         0.0        0.0    0.0   
      43                     0.0         0.0        0.0    0.0   
      45                     0.0         0.0        0.0    0.0   
      47                     0.0         0.0        0.0    0.0   
      49                     0.0         0.0        0.0    0.0   
      50                     0.0         0.0        0.0    0.0   
      51                     0.0         0.0        0.0    0.0   
      53                     0.0         0.0        0.0    0.0   
...                          ...         ...        ...    ...   
test  191299                 0.0         0.0        0.0    0.0   
      191300                 0.0         0.0        0.0    0.0   
      191302                 0.0         0.0        0.0    1.0   
      191303                 0.0         0.0        0.0    0.0   
      191306                 0.0         0.0        0.0    0.0   
      191307                 0.0         0.0        0.0    0.0   
      191308                 0.0         0.0        0.0    0.0   
      191309                 0.0         0.0        0.0    0.0   
      191310                 0.0         0.0        0.0    0.0   
      191314                 0.0         0.0        0.0    0.0   
      191315                 0.0         0.0        0.0    0.0   
      191316                 0.0         0.0        0.0    1.0   
      191317                 0.0         0.0        0.0    0.0   
      191319                 0.0         0.0        0.0    0.0   
      191320                 0.0         0.0        0.0    2.0   
      191321                 0.0         0.0        0.0    0.0   
      191323                 0.0         0.0        0.0    0.0   
      191325                 0.0         0.0        0.0    0.0   
      191327                 0.0         0.0        0.0    0.0   
      191328                 0.0         0.0        0.0    0.0   
      191330                 0.0         0.0        0.0    0.0   
      191332                 0.0         0.0        0.0    0.0   
      191333                 0.0         0.0        0.0    0.0   
      191334                 0.0         0.0        0.0    0.0   
      191336                 0.0         0.0        0.0    0.0   
      191338                 0.0         0.0        0.0    0.0   
      191339                 0.0         0.0        1.0    0.0   
    

In [8]:
allFeature['FN']

DepartmentDescription  FN-1  FN-2  FN0  FN1  FN10  FN100  FN1000  FN1001  \
      VisitNumber                                                          
train 5                 0.0   0.0  0.0  0.0   0.0    0.0    -1.0     0.0   
      7                 0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      8                 0.0   1.0  0.0  0.0   0.0    0.0     0.0     3.0   
      9                 0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      10                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      11                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      12                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      15                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      17                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      19                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      20                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      23                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      25                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      26                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      28                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      29                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      30                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      31                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      32                0.0   0.0  0.0  0.0   0.0    1.0     0.0     0.0   
      33                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      40                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      41                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      42                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      43                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      45                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      47                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      49                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      50                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      51                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      53                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
...                     ...   ...  ...  ...   ...    ...     ...     ...   
test  191299            0.0   0.0  0.0  0.0   0.0    7.0     0.0     0.0   
      191300            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191302            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191303            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191306            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191307            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191308            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191309            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191310            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191314            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191315            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191316            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191317            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191319            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191320            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191321            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191323            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191325            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191327            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191328            0.0   0.0  0.0  0.0   0.

In [9]:
allFeature['DD']

DepartmentDescription  1-HR PHOTO  ACCESSORIES  AUTOMOTIVE  BAKERY  \
      VisitNumber                                                    
train 5                       0.0          0.0         0.0     0.0   
      7                       0.0          0.0         0.0     0.0   
      8                       0.0          0.0         0.0     0.0   
      9                       0.0          0.0         0.0     0.0   
      10                      0.0          0.0         0.0     0.0   
      11                      0.0          0.0         0.0     0.0   
      12                      0.0          0.0         0.0     0.0   
      15                      0.0          0.0         0.0     0.0   
      17                      0.0          0.0         0.0     0.0   
      19                      0.0          1.0         0.0     0.0   
      20                      0.0          0.0         0.0     0.0   
      23                      0.0          0.0         0.0     0.0   
      25                      0.0          0.0         0.0     0.0   
      26                      0.0          0.0         0.0     0.0   
      28                      0.0          0.0         0.0     2.0   
      29                      0.0          0.0         0.0     0.0   
      30                      0.0          0.0         0.0     0.0   
      31                      0.0          0.0         0.0     1.0   
      32                      0.0          0.0         0.0     0.0   
      33                      0.0          0.0         0.0     0.0   
      40                      0.0          0.0         0.0     0.0   
      41                      0.0          0.0         0.0     0.0   
      42                      0.0          0.0         0.0     0.0   
      43                      0.0          0.0         0.0     0.0   
      45                      0.0          0.0         0.0     0.0   
      47                      0.0          0.0         0.0     0.0   
      49                      0.0          0.0         0.0     0.0   
      50                      0.0          0.0         0.0     0.0   
      51                      0.0          0.0         0.0     0.0   
      53                      0.0          0.0         0.0     0.0   
...                           ...          ...         ...     ...   
test  191299                  0.0          0.0         0.0     0.0   
      191300                  0.0          0.0         0.0     0.0   
      191302                  0.0          0.0         0.0     1.0   
      191303                  0.0          0.0         0.0     0.0   
      191306                  0.0          0.0         0.0     0.0   
      191307                  0.0          0.0         0.0     0.0   
      191308                  0.0          0.0         0.0     0.0   
      191309                  0.0          0.0         0.0     0.0   
      191310                  0.0          0.0         0.0     0.0   
      191314                  0.0          0.0         0.0     0.0   
      191315                  0.0          0.0         0.0     0.0   
      191316                  0.0          0.0         0.0     1.0   
      191317                  0.0          0.0         0.0     0.0   
      191319                  0.0          0.0         0.0     0.0   
      191320                  0.0          0.0         0.0     2.0   
      191321                  0.0          0.0         0.0     0.0   
      191323                  0.0          0.0         0.0     0.0   
      191325                  0.0          0.0         0.0     0.0   
      191327                  0.0          0.0         0.0     0.0   
      191328                  0.0          0.0         0.0     0.0   
      191330                  0.0          0.0         0.0     0.0   
      191332                  0.0          0.0         0.0     0.0   
      191333                  0.0          0.0         0.0     0.0   
      191334                  0.0          0.0         0.0     0.0   
      1913

### Save allFeature

In [9]:
def save_feature(allFeature, savePath):
    train_X = sp.sparse.coo_matrix(allFeature.loc['train'].values).tocsr()
    train_y = trainLabel.values.reshape((-1,))
    train_VN = np.array(allFeature.loc['train'].index)
    
    test_X = sp.sparse.coo_matrix(allFeature.loc['test'].values).tocsr()
    test_VN = np.array(allFeature.loc['test'].index)
    
    sp.sparse.save_npz(savePath + 'train_X.npz', train_X)
    np.save(savePath + 'train_y.npy', train_y)
    sp.sparse.save_npz(savePath + 'test_X.npz', test_X)
    np.save(savePath + 'train_VN.npy', train_VN)
    np.save(savePath + 'test_VN.npy', test_VN)

In [10]:
savePath = './origin/'
save_feature(allFeature, savePath)

### Drop Some FLN

In [11]:
k = allFeature.FN.apply(lambda x: x.sum())
dropedFN = [('FN',x) for x in k[k<200].index]

In [12]:
allFeature.drop(dropedFN, axis=1, inplace=True)

In [14]:
allFeature['FN']

DepartmentDescription  FN-1  FN-2  FN0  FN1  FN10  FN100  FN1000  FN1001  \
      VisitNumber                                                          
train 5                 0.0   0.0  0.0  0.0   0.0    0.0    -1.0     0.0   
      7                 0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      8                 0.0   1.0  0.0  0.0   0.0    0.0     0.0     3.0   
      9                 0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      10                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      11                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      12                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      15                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      17                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      19                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      20                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      23                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      25                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      26                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      28                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      29                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      30                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      31                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      32                0.0   0.0  0.0  0.0   0.0    1.0     0.0     0.0   
      33                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      40                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      41                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      42                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      43                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      45                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      47                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      49                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      50                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      51                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      53                0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
...                     ...   ...  ...  ...   ...    ...     ...     ...   
test  191299            0.0   0.0  0.0  0.0   0.0    7.0     0.0     0.0   
      191300            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191302            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191303            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191306            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191307            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191308            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191309            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191310            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191314            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191315            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191316            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191317            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191319            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191320            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191321            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191323            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191325            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191327            0.0   0.0  0.0  0.0   0.0    0.0     0.0     0.0   
      191328            0.0   0.0  0.0  0.0   0.

In [13]:
savePath = './droped/'
save_feature(allFeature, savePath)

### PCA for FinlineNumber (given up)

In [12]:
FN_PCA = 200

if FN_PCA is not None:
    ReducedFN = PCA(n_components=FN_PCA).fit_transform(allFeature['FN'].values)
    allFeature.drop('FN', axis=1, inplace=True)
    multi = pd.MultiIndex.from_tuples([('ReducedFN', 'RFN' + str(x)) for x in range(FN_PCA)])
    ReducedFN = pd.DataFrame(ReducedFN, index=allFeature.index, columns=multi)
    allFeature = pd.concat([allFeature, ReducedFN], axis=1)

In [16]:
savePath = './reduced/'
save_feature(allFeature, savePath)